In [1]:
%load_ext autoreload
%autoreload 2

from litdata import StreamingDataset

from lobster.tokenization._ume_tokenizers import (
        _make_ume_tokenizers,
        UmeAminoAcidTokenizerFast,
        UmeSmilesTokenizerFast,
        UmeNucleotideTokenizerFast,
        UmeLatentGenerator3DCoordTokenizerFast,
    )
from lobster.tokenization._nucleotide_tokenizer import (
    _make_nucleotide_tokenizer,
)

/Users/zadorozk/Desktop/code/lobster/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_make_ume_tokenizers()

In [3]:
# _make_nucleotide_tokenizer()
_make_ume_tokenizers()

tokenizers = [
            UmeAminoAcidTokenizerFast(),
            UmeSmilesTokenizerFast(),
            UmeNucleotideTokenizerFast(),
            UmeLatentGenerator3DCoordTokenizerFast(),
    ]

vocab = {
        token_id: token for tokenizer in tokenizers
        for token, token_id in tokenizer.get_vocab().items()
        if "reserved" not in token
    }

print(f"Total vocabulary size = {len(vocab)}")  # 1536
print(f"Vocab size % 64 = {len(vocab) % 64}")  # 0

Total vocabulary size = 1536
Vocab size % 64 = 0


In [2]:
from lobster.datasets import AMPLIFYIterableDataset
import torch.utils.data

ds = AMPLIFYIterableDataset()
ds_item = next(iter(ds))
sequences = []

for i in range(100):
        
    sequences.append(next(iter(ds)))


In [ ]:
import torch
from lobster.model import Ume

model = Ume()
model.to(torch.device("cuda"))

model.training_step(
    {
        "input_ids": torch.randint(0, 100, (4, 2, 8192)),
        "attention_mask": torch.ones((4, 2, 8192)),
        "modality": [("smiles"), ("amino_acid"), ("nucleotide", "smiles")] * 2
    },
    0
)

['...']